In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [45]:
from proj1_helpers import *

DATA_TRAIN_PATH = '../../../Data-Project1/train.csv' # TODO: add a file Data-Project1 with the train data 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [27]:
from helpers import *

# Standardize the input data
stx, mean_stx, std_x = standardize(tX)

In [43]:
# TODO: Get rid of the -999 in the data

## Do your thing crazy machine learning thing here :) ...

In [44]:
from costs import *

def compute_gradient(y, tx, w):
    """Compute the gradient."""
    e = y - tx @ w
    return -(tx.T @ e)/len(y)

def least_squares_GD(y, tx, gamma, max_iters):
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    w = np.zeros(tx.shape[1])
    ws = [np.copy(w)]
    losses = []
    for n_iter in range(max_iters):
        # compute gradient with mse
        grad = compute_gradient(y, tx, w)
        
        # update w
        w = w - (gamma*grad)
        
        # calculate the cost with mse
        loss = calculate_mse(y - tx @ w)
        
        # store w and loss
        ws.append(np.copy(w))
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses, ws

gamma = 0.4
max_iters = 100
least_squares_GD(y, stx, gamma, max_iters)

Gradient Descent(0/99): loss=0.6277028366519332, w0=-0.1258656, w1=0.09080041296419909
Gradient Descent(1/99): loss=3.9469199851098278, w0=-0.2013849599999991, w1=0.036038577931864074
Gradient Descent(2/99): loss=54.89031966669451, w0=-0.2466965760000015, w1=0.2863556192844112
Gradient Descent(3/99): loss=832.6064224961452, w0=-0.273883545599993, w1=-0.6985479663561021
Gradient Descent(4/99): loss=12703.667939488594, w0=-0.29019572736002813, w1=3.132020511674238
Gradient Descent(5/99): loss=193902.65673310886, w0=-0.29998303641589374, w1=-11.853337039693308
Gradient Descent(6/99): loss=2959709.7700175294, w0=-0.30585542185002534, w1=46.67374980594344
Gradient Descent(7/99): loss=45176771.56347692, w0=-0.30937885310813984, w1=-182.00409346737487
Gradient Descent(8/99): loss=689574675.1290365, w0=-0.3114929118723606, w1=711.4026065152887
Gradient Descent(9/99): loss=10525613481.859308, w0=-0.3127613470941787, w1=-2779.0688519754285
Gradient Descent(10/99): loss=160662134522.67603, w0=-0.

([0.62770283665193316,
  3.9469199851098278,
  54.890319666694509,
  832.60642249614523,
  12703.667939488594,
  193902.65673310886,
  2959709.7700175294,
  45176771.56347692,
  689574675.12903655,
  10525613481.859308,
  160662134522.67603,
  2452334157493.3096,
  37432235279876.039,
  571362688794575.63,
  8721235045294708.0,
  1.3312024429831542e+17,
  2.0319369160454124e+18,
  3.1015324923351044e+19,
  4.7341547491208875e+20,
  7.2261764930760742e+21,
  1.1029978840210589e+23,
  1.6836072760202553e+24,
  2.5698448754360533e+25,
  3.9225909616023033e+26,
  5.9874119247891586e+27,
  9.1391383674793139e+28,
  1.3949908766779003e+30,
  2.1293030784381285e+31,
  3.2501514351430098e+32,
  4.9610055319651955e+33,
  7.5724397399059508e+34,
  1.1558512330824258e+36,
  1.7642822114220679e+37,
  2.6929864609301065e+38,
  4.1105533070626796e+39,
  6.2743161673260939e+40,
  9.5770667418252285e+41,
  1.461835918550188e+43,
  2.2313348234599497e+44,
  3.4058918865004683e+45,
  5.1987265293257793e

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)